## Veri Ön İşleme Notebook'u

## Proje Tanıtımı
Bu notebook, 2023-2024 Bundesliga sezonuna ait haftalık maç verilerini birleştirme, temizleme 
ve efficiency (şut isabet oranı) hesaplama adımlarını göstermektedir.

## WebScraping yöntemi ile verilerin çekilmesi
Githuba yüklediğim scrapeStatsAndSave.js ile her hafta ve sezon için veriler birleştirildi

## 2. Verilerin Birleştirilmesi

In [ ]:
import pandas as pd
import os

data_dir = "data"
all_weeks = []
for week in range(1, 35):
    filename = f"week{week}.xlsx"
    path = os.path.join(data_dir, filename)
    df_week = pd.read_excel(path)
    df_week["Week"] = week
    all_weeks.append(df_week)

full_data = pd.concat(all_weeks, ignore_index=True)
full_data.to_csv("bundesliga_full.csv", index=False)
full_data.head()

# Üzerine yazarak kaydet
full_data.to_csv("bundesliga_full.csv", index=False)
print("Birleştirme tamamlandı, bundesliga_full.csv kaydedildi.")

Bu adımda 34 haftalık Excel dosyaları ve 10 sezon tek bir CSV dosyasında birleştirilmiştir.


## 3. Veri Temizleme 


In [ ]:
import re

df = pd.read_csv("bundesliga_full.csv")

columns_to_split = ["HomePassing Accuracy", "AwayPassing Accuracy",
                    "HomeShots on Target", "AwayShots on Target",
                    "HomeSaves", "AwaySaves"]

def extract_three_parts(val):
    val = str(val).replace('\u00a0', ' ').replace('–', '-').replace('—', '-').replace('−', '-').strip()
    match1 = re.search(r"(\d+)\s*of\s*(\d+)\s*-\s*(\d+)%", val)
    if match1:
        return int(match1.group(1)), int(match1.group(2)), float(match1.group(3))
    match2 = re.search(r"(\d+)%\s*-\s*(\d+)\s*of\s*(\d+)", val)
    if match2:
        return int(match2.group(2)), int(match2.group(3)), float(match2.group(1))
    return 0, 0, 0

for col in columns_to_split:
    success_list, total_list, percent_list = [], [], []
    for val in df[col]:
        success, total, percent = extract_three_parts(val)
        success_list.append(success)
        total_list.append(total)
        percent_list.append(percent)
    df[col + "_Successful"] = success_list
    df[col + "_Total"] = total_list
    df[col + "_Percent"] = percent_list

df.drop(columns=columns_to_split, inplace=True)
df.head()

# Üzerine yazarak kaydet
df.to_csv("bundesliga_full.csv", index=False)
print(" Temizlik tamamlandı, bundesliga_full.csv güncellendi.")

## 4. Yeni Özniteliklerin Türetilmesi (Feature Engineering)

Bu adımda veri setine yeni anlamlı öznitelikler (features) eklenmiştir. Amaç, modelin maç sonucunu daha iyi tahmin edebilmesi için daha anlamlı ve özetleyici bilgileri üretmektir.

In [ ]:
import pandas as pd

# Birleştirilmiş ve temizlenmiş veriyi oku
df = pd.read_excel("bundesliga_full.csv")  

# 1️ Gol farkı
df['GoalDiff'] = df['Home Goal'] - df['Away Goal']

# 2️ xG farkı
df['xGDiff'] = df['Home xG'] - df['Away xG']

# 3️ Hücum verimliliği: atılan gol / xG
df['HomeEfficiency'] = df['Home Goal'] / (df['Home xG'] + 1e-5)
df['AwayEfficiency'] = df['Away Goal'] / (df['Away xG'] + 1e-5)

# 4️ Şut isabet oranı
if 'HomeShots on Target_Successful' in df.columns and 'HomeShots on Target_Total' in df.columns:
    df["HomeShotEfficiency"] = df["HomeShots on Target_Successful"] / (df["HomeShots on Target_Total"] + 1e-5)
    df["AwayShotEfficiency"] = df["AwayShots on Target_Successful"] / (df["AwayShots on Target_Total"] + 1e-5)

# 5️ Kurtarış verimliliği
if 'HomeSaves_Successful' in df.columns and 'HomeSaves_Total' in df.columns:
    df["HomeSaveEfficiency"] = df["HomeSaves_Successful"] / (df["HomeSaves_Total"] + 1e-5)
    df["AwaySaveEfficiency"] = df["AwaySaves_Successful"] / (df["AwaySaves_Total"] + 1e-5)

# 6️ Topa sahip olma farkı
if 'HomePossession' in df.columns and 'AwayPossession' in df.columns:
    df['PossessionDiff'] = df['HomePossession'] - df['AwayPossession']

# 7️ Pas isabet farkı
if 'HomePassing Accuracy_Percent' in df.columns and 'AwayPassing Accuracy_Percent' in df.columns:
    df['PassAccuracyDiff'] = df['HomePassing Accuracy_Percent'] - df['AwayPassing Accuracy_Percent']

# 8️ Faul farkı
if 'HomeFouls' in df.columns and 'AwayFouls' in df.columns:
    df['FoulDiff'] = df['HomeFouls'] - df['AwayFouls']

# 9️ Maç sonucu etiketi
df['Winner'] = df.apply(
    lambda x: 'Home' if x['Home Goal'] > x['Away Goal'] 
              else ('Away' if x['Away Goal'] > x['Home Goal'] else 'Draw'), axis=1
)

df.to_csv("bundesliga_full.csv", index=False)
print(" Feature engineering tamamlandı, bundesliga_full.csv güncellendi.")


## 5. Ölçekleme Ve Normalizasyon

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 1. Veriyi yükle
df = pd.read_csv("bundesliga_full.csv")  

# 2. Hedef değişkeni ayır
y = df["Result"]  # veya "Winner"
X = df.drop(columns=["Result", "Winner"], errors='ignore')

# 3. Sadece sayısal sütunları seç
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns
X_numeric = X[numeric_cols]

# 4. MinMax Ölçekleme (0-1 aralığına)
minmax_scaler = MinMaxScaler()
X_minmax_scaled = pd.DataFrame(minmax_scaler.fit_transform(X_numeric), columns=numeric_cols)

# 5. StandardScaler (z-score)
standard_scaler = StandardScaler()
X_standard_scaled = pd.DataFrame(standard_scaler.fit_transform(X_numeric), columns=numeric_cols)

# 6. Gözlem için çıktı ver
print("🔹 MinMaxScaled veri:")
print(X_minmax_scaled.head())

print("\n🔹 StandardScaled veri:")
print(X_standard_scaled.head())

# Üzerine yazarak kaydet
df.to_csv("bundesliga_full.csv", index=False)
print("✅ Ölçekleme/normalizasyon tamamlandı, bundesliga_full.csv güncellendi.")